In [16]:
import numpy as np
from numpy import nan

from verbose.verbose import print_state, print_experience, print_buffer, print_shapes
from verbose.verbose import print_action_available, print_interaction
from verbose.performance import print_performance, print_episode_duration
from utils.constant import *
from verbose.plot import plot_evolution
from environnement.pre_processing import pre_processing_state, denormalize_vector


from agent.DQN import DQNAgent
from environnement.env import Env
from agent.strategy import EpsilonGreedy


%load_ext autoreload
%autoreload 2


from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
sess = InteractiveSession(config=config)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2022-06-21 09:42:00.638043: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-21 09:42:00.638261: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/widium/.conda/envs/tf_gpu/lib/python3.8/site-packages/tensorflow/python/client/session.py:1771: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [17]:
STACK_SIZE = 3
NBR_STACK = 2

env = Env(STACK_SIZE)
strategy = EpsilonGreedy(MIN_EPSILON, START_EPSILON, DECAY)
agent = DQNAgent(STACK_SIZE, NBR_STACK, len(env.action_space()))

In [18]:
print_state(env, shape=True)

---State t---
-A-	-B-
___________	
|247.0          	xxx|
|893.0          	xxx|
|585.0          	xxx|
___________	
A shape : (3, 1)
B shape : (3, 1)
___________	
------------
Env : in progress
------------


In [20]:
episode_durations = list()
cummulative_reward = list()
epsilon = list()

for episode in range(EPISODE):
    
    print(f"========= Episode : [{episode}] =========")
    env.reset(STACK_SIZE)
    
    state = env.get_state()
    # print_state(env, shape=True)
    
    timesteps = 0
    while (1):
        
        # action_available(env)
        print_state(env)
        timesteps += 1
        
        exploration_rate = strategy.get_exploration_rate(timesteps)
        # print_action_available(env)
        action = env.choose_action(state, agent.policy_model, exploration_rate)
        reward = env.reward()
        next_state = env.get_state()
        print_interaction(exploration_rate, action, reward)
        
        
        
        experience = env.create_experience(state, action, next_state, reward)
        env.replaymemory.push(experience)
        state = next_state
        
        # print_experience(experience)
        if env.replaymemory.can_provide_sample(BATCH_SIZE):
            print("======== START TRAIN ========")
            
            buffer = env.replaymemory.get_sample(BATCH_SIZE)
            # print_buffer(buffer)
            
            states, actions, next_states, rewards, dones = env.replaymemory.extract_value(buffer)
            # print_shapes(states, next_states)
            agent.train(states, next_states, buffer)
        
        # print(f"Env : {env.state()}")
        if (env.state() == 'done'):
            print_state(env)
            print_performance(strategy.epsilon, timesteps, env.cummulative_reward)
            epsilon.append(strategy.epsilon)
            episode_durations.append(timesteps)
            cummulative_reward.append(env.cummulative_reward)            
            break
        
print_episode_duration(episode_durations)
plot_evolution(episode, episode_durations, cummulative_reward)

            
    
    

========= Episode : [0] =========
---State t---
-A-	-B-
___________	
|246.0          	xxx|
|959.0          	xxx|
|270.0          	xxx|
___________	
------------
Env : in progress
------------
Exploration
----------------------------------
Action : [inverse_rotate_b]
Reward : [-5]
Epsilon : [0.9991504248583688]
----------------------------------
---State t---
-A-	-B-
___________	
|246.0          	xxx|
|959.0          	xxx|
|270.0          	xxx|
___________	
------------
Env : in progress
------------
Exploration
----------------------------------
Action : [rotate_b]
Reward : [-10]
Epsilon : [0.9983016988672331]
----------------------------------
---State t---
-A-	-B-
___________	
|246.0          	xxx|
|959.0          	xxx|
|270.0          	xxx|
___________	
------------
Env : in progress
------------
Exploration
----------------------------------
Action : [inverse_rotate_b]
Reward : [-10]
Epsilon : [0.9974538211778671]
----------------------------------
---State t---
-A-	-B-
___________

KeyboardInterrupt: 

In [5]:
env.replaymemory.print_history(BATCH_SIZE)

Actions : 5 	 Reward : 92
Actions : 7 	 Reward : -1
Actions : 5 	 Reward : -1
Actions : 0 	 Reward : -1
Actions : 5 	 Reward : -1
Actions : 3 	 Reward : -6
Actions : 1 	 Reward : -1
Actions : 5 	 Reward : -1
Actions : 0 	 Reward : -1
Actions : 3 	 Reward : -1
Actions : 3 	 Reward : -1
Actions : 7 	 Reward : -1
Actions : 5 	 Reward : -1
Actions : 7 	 Reward : -1
Actions : 5 	 Reward : -1
Actions : 0 	 Reward : -1
Actions : 2 	 Reward : -1
Actions : 2 	 Reward : -1
Actions : 4 	 Reward : -6
Actions : 1 	 Reward : -1
Actions : 0 	 Reward : -6
Actions : 6 	 Reward : -1
Actions : 1 	 Reward : -1
Actions : 0 	 Reward : -6
Actions : 1 	 Reward : -6
Actions : 3 	 Reward : -1
Actions : 1 	 Reward : -1
Actions : 7 	 Reward : -1
Actions : 5 	 Reward : -1
Actions : 0 	 Reward : -1
Actions : 7 	 Reward : -1
Actions : 1 	 Reward : 99
